In [8]:
import torch
import torch.nn as nn
import numpy as np
from torch.autograd import Variable


In [9]:
model = nn.Sequential(
    nn.Linear(1, 64),
    nn.Tanh(),
    nn.Linear(64, 64),
    nn.Tanh(),
    nn.Linear(64, 1),
)

In [10]:
class SineModel(nn.Module):
    
    def __init__(self):
        nn.Module.__init__(self)
        
        self.main = nn.Sequential(
            nn.Linear(1, 40),
            nn.ReLU(),            
            nn.Linear(40, 40),
            nn.ReLU(),
            nn.Linear(40, 1),
        )
        
    def forward(self, x):
        return self.main(x)

class SineWaveTask:
    def __init__(self):
        self.a = np.random.uniform(0.1, 5.0)
        self.b = np.random.uniform(0, 2*np.pi)
        self.train_x = None
        
    def f(self, x):
        return self.a * np.sin(x + self.b)
        
    def training_set(self, size=10, force_new=False):
        if self.train_x is None and not force_new:
            self.train_x = np.random.uniform(-5, 5, size)
            x = self.train_x
        elif not force_new:
            x = self.train_x
        else:
            x = np.random.uniform(-5, 5, size)
        y = self.f(x)
        return torch.Tensor(x), torch.Tensor(y)
    
    def test_set(self, size=50):
        x = np.linspace(-5, 5, size)
        y = self.f(x)
        return torch.Tensor(x), torch.Tensor(y)
    



In [13]:
def do_base_learning(model, wave, lr_inner, n_inner):
    new_model = SineModel()
    inner_optimizer = torch.optim.SGD(new_model.parameters(), lr=lr_inner)
    # K steps of gradient descent
    for i in range(n_inner):

        x, y_true = wave.training_set()
        x = Variable(x[:, None])
        y_true = Variable(y_true[:, None])

        y_pred = new_model(x)

        loss = ((y_pred - y_true)**2).mean()

        inner_optimizer.zero_grad()
        loss.backward()
        inner_optimizer.step()
    return new_model

In [17]:
import random 
TRAIN_SIZE = 10000
TEST_SIZE = 1000
SINE_TRAIN = [SineWaveTask() for _ in range(TRAIN_SIZE)]
wave = random.sample(SINE_TRAIN, 1)[0]

        # Take k gradient steps
new_model = do_base_learning(model, wave, 0.001, 10000)

In [36]:
params = new_model.named_parameters()
params

<generator object Module.named_parameters at 0x000001E9BE108820>

In [38]:
for name, weight in params:
    print(name,weight.size())

main.0.bias torch.Size([40])
main.2.weight torch.Size([40, 40])
main.2.bias torch.Size([40])
main.4.weight torch.Size([1, 40])
main.4.bias torch.Size([1])
